In [1]:
"https://api.scryfall.com/cards/search?order=set&unique=art&q=set%3A'eld'+lang%3A'en'"

"https://api.scryfall.com/cards/search?order=set&unique=art&q=set%3A'eld'+lang%3A'en'"

In [2]:
# Change dir to get SQL connector
import os
os.chdir("..")
from src.sqldb import SQLDatabase

In [3]:
from src import models as m

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
import seaborn as sns

# Updating Tables

In [5]:
FLAT_FILE_DIR = "flat_files/"

full_table = pd.read_csv(FLAT_FILE_DIR + "full_table.csv")
event_table = pd.read_csv(FLAT_FILE_DIR + "event_table.csv")
deck_list_table = pd.read_csv(FLAT_FILE_DIR + "deck_list_table.csv")
pilot_table = pd.read_csv(FLAT_FILE_DIR + "pilot_table.csv")
deck_table = pd.read_csv(FLAT_FILE_DIR + "deck_table.csv")
card_table = pd.read_csv(FLAT_FILE_DIR + "card_table.csv")

In [5]:

m.update_cards()

OperationalError: table card has no column named image_uris

In [34]:
with SQLDatabase() as sql:
    df = sql.get_dataframe_from("card")

df.to_csv(FLAT_FILE_DIR + "card_table.csv")

In [37]:
df

,setNumber,setName,name,cmc,color,mana_cost,standardLegality,oracle_text,image_uri
0,1,eld,Acclaimed Contender,3,W,{2}{W},legal,When Acclaimed Contender enters the battlefiel...,None
1,2,eld,All That Glitters,2,W,{1}{W},legal,Enchant creature\nEnchanted creature gets +1/+...,None
2,3,eld,Archon of Absolution,4,W,{3}{W},legal,Flying\nProtection from white (This creature c...,None
3,4,eld,Ardenvale Paladin,4,W,{3}{W},legal,Adamant — If at least three white mana was spe...,None
4,5,eld,Ardenvale Tactician // Dizzying Swoop,3,W,{1}{W}{W} // {1}{W},legal,Flying // Tap up to two target creatures. (The...,None
...,...,...,...,...,...,...,...,...,...
14482,378,stx,Fracture,2,BW,{W}{B},legal,"Destroy target artifact, enchantment, or plane...",None
14483,379,stx,Expressive Iteration,2,RU,{U}{R},legal,Look at the top three cards of your library. P...,None
14484,380,stx,Mortality Spear,4,BG,{2}{B}{G},legal,This spell costs {2} less to cast if you gaine...,None
14485,381,stx,Rip Apart,2,RW,{R}{W},legal,Choose one —\n• Rip Apart deals 3 damage to ta...,None


In [35]:
card_table

,Unnamed: 0,setNumber,setName,name,cmc,color,mana_cost,standardLegality,oracle_text
0,0,1,eld,Acclaimed Contender,3,W,{2}{W},legal,When Acclaimed Contender enters the battlefiel...
1,1,2,eld,All That Glitters,2,W,{1}{W},legal,Enchant creature\nEnchanted creature gets +1/+...
2,2,3,eld,Archon of Absolution,4,W,{3}{W},legal,Flying\nProtection from white (This creature c...
3,3,4,eld,Ardenvale Paladin,4,W,{3}{W},legal,Adamant — If at least three white mana was spe...
4,4,5,eld,Ardenvale Tactician // Dizzying Swoop,3,W,{1}{W}{W} // {1}{W},legal,Flying // Tap up to two target creatures. (The...
...,...,...,...,...,...,...,...,...,...
14482,14482,378,stx,Fracture,2,BW,{W}{B},legal,"Destroy target artifact, enchantment, or plane..."
14483,14483,379,stx,Expressive Iteration,2,RU,{U}{R},legal,Look at the top three cards of your library. P...
14484,14484,380,stx,Mortality Spear,4,BG,{2}{B}{G},legal,This spell costs {2} less to cast if you gaine...
14485,14485,381,stx,Rip Apart,2,RW,{R}{W},legal,Choose one —\n• Rip Apart deals 3 damage to ta...


# Cleaning

New decks need to be checked for misformed codes & missing archetypes/categories

In [8]:
for idx, row in deck_table.iterrows():
    print(row["name"])

Gruul Aggro
UW Control
Red Deck Wins
Rakdos Aggro
Gruul Aggro
Gruul Aggro
Gruul Aggro
Rakdos Aggro
Boros Cycling Aggro
Gruul Aggro
Gruul Aggro
Rakdos Control
Rakdos Control
Gruul Aggro
Gruul Aggro
Mono Black Aggro
Gruul Aggro
Selesnya Aggro
Dimir Rogue
Gruul Aggro
Dimir Flash
Gruul Aggro
Gruul Aggro
Rakdos Aggro
UW Control
Rakdos Aggro
Rakdos Control
Rakdos Aggro
Abzan DOOM Yorion
Gruul Aggro
Dimir Mill
Gruul Aggro
Gruul Aggro
Rakdos Control
Gruul Aggro
Gruul Aggro
Gruul Aggro
UW Control
Izzet Control
Sultai Mutate Aggro
Izzet Control Kick
Mono Black Aggro
Dimir Rogue
Rakdos Aggro
Dimir Mill
Esper Yorion
Gruul Aggro
Gruul Aggro
Mono Blue Aggro
Rakdos Aggro
Gruul Aggro
Gruul Aggro
Selesnya Aggro
UW Yorion
Gruul Aggro
Gruul Aggro
Dimir Mill
Mono Black Aggro
Mono Green Aggro
Red Deck Wins
Red Deck Wins
UB Mill
Rakdos Aggro
Rakdos Aggro
UB Control
UB Mill
UB Mill
Gruul Aggro
Gruul Adventure Aggro
Gruul Aggro
UB Control
Selesnya Aggro
UB Mill
Rakdos Aggro
Gruul Aggro
Mono Black Aggro
Esper 

Temur Ramp Ultimatum 	
Mono Green Aggro
Bant Yorion
Gruul Aggro
Dimir Mill
Rakdos Aggro
Dimir Rogue
Esper DOOM Yorion
Gruul Aggro
Gruul Aggro
Gruul Aggro
Red Deck Wins
Esper DOOM Yorion
Temur Ramp Ultimatum
Abzan DOOM Control
Esper DOOM Yorion
Gruul Aggro
Gruul Aggro
Rakdos Aggro
Temur Ramp Ultimatum
Esper DOOM Yorion
Rakdos Aggro
Dimir Mill
Dimir Rogue
Dimir Rogue
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Gruul Aggro
Gruul Aggro
Gruul Aggro
Gruul Aggro
Gruul Aggro
Gruul Aggro
Gruul Aggro
Gruul Aggro
Gruul Aggro
Gruul Aggro
Gruul Aggro
Izzet Control
Jeskai Control
Mono Green Aggro
Mono Green Aggro
UB Control
UB Control
Azorius Aggro
Boros Cycling Aggro
Dimir Mill
Dimir Rogue
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Esper DOOM Yorion
Gruul Adventure Aggro
Gruul Aggro
Gruul Aggro
Gruul 

Mono Green Aggro
Gruul Aggro
Gruul Aggro
Mono Green Aggro
Mono Green Aggro
Dimir Control
Esper DOOM Yorion
Gruul Aggro
Rakdos Control
Boros Cycling Aggro
Esper DOOM Yorion
Gruul Aggro
Rakdos Control
Gruul Aggro
Sultai Control
Dimir Mill
Gruul Aggro
Dimir Control
Dimir Mill
Gruul Aggro
Jund
Mono Green Aggro
Mono Green Aggro
Gruul Aggro
Gruul Aggro
Boros Control
Boros Control
Gruul Aggro
Mono Green Aggro
Dimir Mill
Boros Cycling Aggro
Dimir Control
Gruul Aggro
Dimir Mill
Jund
Dimir Mill
Gruul Aggro
Gruul Aggro
Temur Ramp Ultimatum
Gruul Aggro
Dimir Control
Gruul Aggro
Gruul Aggro
Dimir Control
Dimir Control
Gruul Aggro
Gruul Aggro
Gruul Aggro
Dimir Control
Esper DOOM Yorion
Gruul Aggro
Dimir Mill
Gruul Aggro
Gruul Aggro
Weenie White
Red Deck Wins
Gruul Aggro
Dimir Mill
Gruul Aggro
Dimir Control
Dimir Mill
Mono Green Aggro
Selesnya Aggro
Dimir Rogue
Jund
Dimir Control
Gruul Aggro
Dimir Mill
Dimir Rogue
Gruul Aggro
Gruul Aggro
Dimir Control
Gruul Aggro
Dimir Control
Gruul Aggro
Gruul Aggro

Rakdos Aggro
4c Yorion
Rakdos Aggro
Runes Aggro
Temur Yorion Ultimatum
Boros Cycling Aggro
Red Deck Wins
Sultai Yorion Ultimatum
Temur Aggro Adventure
Dimir Mill
Naya Aggro
Naya Aggro
Temur Aggro Adventure
Temur Aggro Adventure
Temur Aggro Adventure
Naya Aggro
Red Deck Wins
Boros Cycling Aggro
Boros Cycling Aggro
Naya Aggro
Weenie White
Jund
Mono Green Aggro
Weenie White
Weenie White
Dimir Mill
Abzan Yorion
Naya Aggro
Boros Cycling Aggro
Weenie White 
Naya Aggro
Boros Cycling Aggro
Naya Aggro
Temur Aggro Adventure
Red Deck Wins
Red Deck Wins
Sultai Yorion Ultimatum
Jeskai Control
Temur Aggro Adventure
Rakdos Aggro
Naya Aggro
Temur Aggro Adventure
Temur Aggro Adventure
Red Deck Wins
Temur Aggro Adventure
Boros Cycling Aggro
Naya Aggro
Temur Aggro Adventure
UW Control
Temur Aggro Adventure
Naya Aggro
Sultai Control
Dimir Control
Izzet Control
Izzet Control
Red Deck Wins
Naya Aggro
Naya Aggro
Rakdos Sacrifice
Gruuldown
Mono W Agro
Sultai Valki
UW Control
Naya Aggro
Sultai Yorion Ultimatum

Dimir Mill
Dimir Mill
Temur Aggro Adventure
Naya Aggro
Jeskai Cycling Aggro
Naya Aggro
Dimir Mill
Gruul Aggro
Jeskai Cycling Aggro
Temur Aggro Adventure
Gruul Aggro
Sultai Yorion Ultimatum
Jeskai Cycling Aggro
Temur Aggro Adventure
Red Deck Wins
Red Deck Wins
Sultai Yorion Ultimatum
Temur Aggro Adventure
Sultai Yorion Ultimatum
Naya Aggro
Sultai Yorion
Temur Aggro Adventure
Dimir Mill
Rakdos Aggro
Red Deck Wins
Sultai Yorion Ultimatum
4c Yorion
Gruul Aggro
Red Deck Wins
Temur Aggro Adventure
Dimir Rogue Mill
Naya Aggro
Red Deck Wins
Temur Aggro Adventure
Weenie White 
Gruul Aggro
Dimir Mill
Sultai Yorion Ultimatum
Weenie White
Jeskai Cycling Aggro
Sultai Yorion
Red Deck Wins
Rakdos Aggro
Jeskai Yorion
Naya Aggro
Mono Green Aggro
Naya Aggro
Rakdos Aggro
UR Aggro
Dimir Mill
Sultai Midrange
Dimir Rogue Mill
4c Sanctum
Golgari Aggro
Sultai Yorion
Gruul Aggro
Dimir Mill
Naya Aggro
Jeskai Cycling Aggro
Temur Lukka
Boros Aggro
Red Deck Wins
Sultai Yorion Ultimatum
Gruul Aggro
Red Deck Wins
Re

In [9]:
deck_table.loc[deck_table.archetype.isna()].empty

True

In [10]:
deck_table.loc[deck_table.category.isna()]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category


In [11]:
unmatched = set(deck_list_table.loc[~(deck_list_table.cardId.isin(card_table.cardId))]["cardId"])

def find_urls_with_unmatched(unmatched):
    unmatched_deck_ids = list(deck_list_table.loc[deck_list_table.cardId == unmatched]["deckId"])
    return set(deck_table.loc[deck_table.deckId.isin(unmatched_deck_ids)]["deckUrl"])

def print_page_source(unmatched):
    unmatched_url = find_urls_with_unmatched(unmatched).pop()
    print(unmatched_url)
    html = requests.get("https://www.mtgtop8.com/event" + unmatched_url)
    return BeautifulSoup(html.text, features="lxml")

AttributeError: 'DataFrame' object has no attribute 'cardId'

In [12]:
unmatched

NameError: name 'unmatched' is not defined

In [18]:
standard_sets = "eld|thb|znr|iko|khm|m21|stx"
names = full_table[
    (~full_table["cardId"].str.contains(standard_sets, regex=True))
    & (full_table["date"] >= "2020-09-29")
][["cardId", "name"]].drop_duplicates().reset_index(drop=True)["name"]

In [19]:
names

0    Midnight Reaper
1             Murder
Name: name, dtype: object

In [20]:
if (names == pd.Series(["Midnight Reaper", "Murder"])).all():
    print("do a thing")

do a thing


In [21]:
deck_table[
    deck_table["deckId"].isin(
        full_table[
            full_table["cardId"] == "101m13"
        ]
        ["deckId"]
    )
]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category


In [17]:
try:
    has_points = deck_table[deck_table["rank"].str.contains("pts")]
except AttributeError:
    has_points = "No Points"
has_points

'No Points'

In [26]:
deck_list_table[deck_list_table["cardName"].isna()]

,cardId,deckId,count,slot,cardName
